# Data preparation




In [1]:
PATH = "/content/drive/MyDrive/MSc /Narratives Project Nadav/analysis/data/raw"

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive/MSc /Narratives Project Nadav/analysis/data/raw"

storeleads-scraping-about_data_toclean.csv  storeleads-scraping-about_data_toclean_sample5.csv


In [4]:
import os
import pandas as pd
import numpy as np

data_path = os.path.join(PATH, "storeleads-scraping-about_data_toclean_sample5.csv")
df = pd.read_csv(data_path)



# add empty row called clean text
df['clean_text'] = ''
df['purged_text'] = ''
display(df.head())

,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
0,https://crafthq.co.uk/pages/about-us,Free delivery on orders over £40Free delivery ...,0.0,7,crafthq.co.uk,crafthq.co.uk/pages/about-us,US,en,shopify,,
1,https://active-silver.co.uk/pages/about-active...,Country/regionCountry/region ...,5.0,7,active-silver.co.uk,active-silver.co.uk/pages/about-active-silver,US,en,shopify,,
2,https://smartstore24.org/pages/about-us,About Us Summer Sale - 50% OFF + FREE Shipping...,1.0,4,smartstore24.org,smartstore24.org/pages/about-us,CN,en,shopify,,
3,https://www.kbeautyshop.com.co/quienes-somos,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,1.0,17,kbeautyshop.com.co,kbeautyshop.com.co/quienes-somos,CO,es,tiendanube,,
4,https://www.infinitymarts.com.br/pages/quem-somos,Quem Somos Frete Grátis para todo o Brasil ...,1.0,13,infinitymarts.com.br,infinitymarts.com.br/pages/quem-somos,BR,pt,shopify,,


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb 12 16:43:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [7]:

!pip install transformers torch sentencepiece -q  # or accelerate if you want GPU acceleration


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [22]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move the model to GPU if available/desired
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [23]:
def clean_text_with_llm(text: str) -> str:
    """
    Use an LLM to clean or rewrite the given text.
    The prompt is just an example. Adjust it to your needs!
    """
    if not isinstance(text, str) or not text.strip():
        return ""  # handle empty or invalid entries

    # Prompt style: We ask the model to "remove spam or unwanted content" from the text
    prompt = f"Remove any spam or unwanted advertisements and return the clean text:\n\n{text}\n\nCleaned text:"

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,         # adjust as needed
        num_beams=1,                # or higher for better but slower results
        do_sample=False             # or True for sampling
    )
    # Decode
    cleaned = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return cleaned.strip()


In [24]:
def llm_clean_dataframe(df, text_col="text"):
    # Prepare columns in case they're missing
    if "clean_text" not in df.columns:
        df["clean_text"] = ""
    if "purged_text" not in df.columns:
        df["purged_text"] = ""

    for i, row in df.iterrows():
        original_text = row[text_col]
        # We call our LLM-based cleaning function
        cleaned_text = clean_text_with_llm(original_text)

        # Store results
        df.at[i, "clean_text"] = cleaned_text
        # Optionally set purged_text to "" or some placeholder
        df.at[i, "purged_text"] = ""

    return df


In [25]:
# Example usage:
df_ex = df.head(10)

df_ex = llm_clean_dataframe(df_ex, text_col="text")

# Save to CSV
df_ex.to_csv("llm_cleaned.csv", index=False)

print("Done. Check llm_cleaned.csv for results.")


Done. Check llm_cleaned.csv for results.


In [28]:
display(df_ex)

# 2) Write desired output to a text file
with open("output.txt", "w", encoding="utf-8") as f:
    for i, r in df_ex.iterrows():
        print(r["clean_text"] + "\n")  # Already contains \n between segments

for i, r in df_ex.iterrows():
    print(r["clean_text"] + "\n+++++++++++++++++++++++++++++++++\n")  # Already contains \n between segments


,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
0,https://crafthq.co.uk/pages/about-us,Free delivery on orders over £40Free delivery ...,0.0,7,crafthq.co.uk,crafthq.co.uk/pages/about-us,US,en,shopify,Free delivery on orders over £40Free delivery ...,
1,https://active-silver.co.uk/pages/about-active...,Country/regionCountry/region ...,5.0,7,active-silver.co.uk,active-silver.co.uk/pages/about-active-silver,US,en,shopify,Colloidal Silver is a liquid made of nano size...,
2,https://smartstore24.org/pages/about-us,About Us Summer Sale - 50% OFF + FREE Shipping...,1.0,4,smartstore24.org,smartstore24.org/pages/about-us,CN,en,shopify,About Us Summer Sale - 50% OFF + FREE Shipping...,
3,https://www.kbeautyshop.com.co/quienes-somos,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,1.0,17,kbeautyshop.com.co,kbeautyshop.com.co/quienes-somos,CO,es,tiendanube,"K-Beauty Shop, tu destino online para descubri...",
4,https://www.infinitymarts.com.br/pages/quem-somos,Quem Somos Frete Grátis para todo o Brasil ...,1.0,13,infinitymarts.com.br,infinitymarts.com.br/pages/quem-somos,BR,pt,shopify,- Criptografia de ponta a ponta (SSL) - Certif...,
5,https://httpalirabexpress.com/pages/about-us,"About Us Launched In 2015, Ali Rub Express Is ...",0.0,7,httpalirabexpress.com,httpalirabexpress.com/pages/about-us,US,en,shopify,"About Us Launched In 2015, Ali Rub Express Is ...",
6,https://gigiszone.com/pages/about-us,About us Awkwardness gives me great comfort. I...,2.0,81,gigiszone.com,gigiszone.com/pages/about-us,US,en,shopify,Awkwardness gives me great comfort. I’ve never...,
7,https://minisan.com/pages/about,"SUMMER SALES -30% | FREE SHIPPING OVER €29,90 ...",5.0,14,minisan.com,minisan.com/pages/about,US,en,shopify,"Konbiniverse®, a global community of people wh...",
8,https://chilithom.com/pages/about-chili-thom,CartYour cart is empty. Continue browsing ...,4.0,10,chilithom.com,chilithom.com/pages/about-chili-thom,SG,en,shopify,Wild).,
9,https://lushvirginhair.com/pages/about-us,About Us Your cart is currently empty!Continue...,0.0,13,lushvirginhair.com,lushvirginhair.com/pages/about-us,US,en,shopify,"LVH hair is the place to shop your wig, we pro...",


Free delivery on orders over £40Free delivery on orders over £40For Candle Makers by Candle MakersHere at Craft HQ we thoroughly test all fragrance oils to ensure they perform as well as can be in candles, wax melts and lots of other fragranced products.Starting your own candle and wax melt business is hard work. Not only have you got to perfect your craft and create the best candles and wax melts possible, but you also need to market yourself and get customers! As owners of a successful UK based home fragrance brand, we know what it takes. We have created Craft HQ to help other candle and wax melt businesses start, grow and thrive through the use of our unique fragrance oils and marketing advice.This site is protected by reCAPTCHA and the Google Privacy Policy and Terms of Service apply.Unit 7 Boss Hall RoadIpswich, IP1 5BNMon - Fri, 8:00am -

Colloidal Silver is a liquid made of nano sized silver particles, suspended in ultra-pure water. This natural product can help with a huge vari

In [ ]:
text = """About FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club. Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner. We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession. Community focused. Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia. We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams. We don't do normal. We overdeliver. From the way we teach, to the tools we provide, we believe in standing out and doing things differently. We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods. We love good humans. We believe in treating people with respect and kindness. From strangers on the Internet, to our SRNAs and fellow colleagues, we practice the Golden Rule. We value transparency and doing the right thing, even when nobody is looking. We promise to bring inclusivity, openness and acceptance to every interaction. For the CRNA school applicant, helpful articles and resources. Check out our most popular applicant resource, The CRNA School Database. Applying to CRNA school? Our digital courses will take you further, faster. And that's why we founded The CRNA Club.    """
label_1 = "ad" # @param {"type":"string","placeholder":"ad"}
label_2 = "about us" # @param {"type":"string","placeholder":"about us"}
labels = []
labels.append(label_1)
labels.append(label_2)

import nltk
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# 4. Split text into segments (paragraphs or lines)
# Split into sentences or paragraphs:
segments = sent_tokenize(text)

print(f"Text split into {len(segments)} segments.")
# Prepare lists to store the two categories
spammy_segments = []
informative_segments = []

for seg in segments:
    # Clean up the segment (strip whitespace)
    seg = seg.strip()
    if not seg:
        continue  # skip empty lines

    # 5. Classify the segment using zero-shot classification
    result = classifier(seg, labels)

    print(f"Segment: {seg}")
    print(f"Classification result: {result['scores']}")


    # The pipeline's result["labels"] might reorder the labels by descending probability
    # so we find the index of each label in result["labels"]
    spammy_score = result["scores"][result["labels"].index(labels[0])]
    informative_score = result["scores"][result["labels"].index(labels[1])]

    # 6. Decide which category is higher
    if spammy_score > informative_score:
        spammy_segments.append(seg)
    else:
        informative_segments.append(seg)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Text split into 23 segments.


/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:167: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


Segment: About FLASH SALE!
Classification result: [0.9698034524917603, 0.03019654005765915]
Segment: 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
Classification result: [0.8721286058425903, 0.12787143886089325]
Segment: 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
Classification result: [0.6727341413497925, 0.32726579904556274]
Segment: And that's why we founded The CRNA Club.
Classification result: [0.9912635087966919, 0.00873648002743721]
Segment: Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
Classification result: [0.9968127012252808, 0.0031873127445578575]
Segment: We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
Classification result: [0.9923029541969299, 0.00769706629216671]
Segment: Community focused.
Classification result: [0.9829450249671936, 0.01705498993396759]
Segment: 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Segment: We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
Classification result: [0.9190756678581238, 0.08092434704303741]
Segment: We don't do normal.
Classification result: [0.9995747208595276, 0.00042532241786830127]
Segment: We overdeliver.
Classification result: [0.9976714253425598, 0.0023286191280931234]
Segment: From the way we teach, to the tools we provide, we believe in standing out and doing things differently.
Classification result: [0.972650408744812, 0.027349654585123062]
Segment: We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods.
Classification result: [0.9906060695648193, 0.00939391553401947]
Segment: We love good humans.
Classification result: [0.9193246960639954, 0.08067528158426285]
Segment: We believe in treating people with respect and kindness.
Classification result: [0.9950705170631409, 0.004929

In [ ]:
# 7. Print or otherwise use the results
print("===== SPAMMY SEGMENTS =====")
for s in spammy_segments:
    print(f"- {s}")

print("\n===== INFORMATIVE SEGMENTS =====")
for s in informative_segments:
    print(f"- {s}")

===== SPAMMY SEGMENTS =====
- About FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
- For the CRNA school applicant, helpful articles and resources.
- Check out our most popular applicant resource, The CRNA School Database.

===== INFORMATIVE SEGMENTS =====
- And that's why we founded The CRNA Club.
- Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
- We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
- Community focused.
- Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.
- We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
- We don't do normal.
- We overdeliver.
- From the w

In [ ]:
import math
def classify_subchunk(sent_list,labels):
    """
    Returns (best_label, best_score_log) for the entire subchunk.
    best_score_log = sum of log(probabilities) for the chosen label across all sentences.
    """
    # We'll sum up the log probabilities for each label across all sentences,
    # then pick which label has the higher total.
    total_ad_log = 0.0
    total_story_log = 0.0

    for s in sent_list:
        s = s.strip()
        if not s:
            continue
        result = classifier(s, labels)
        # e.g. result["labels"] = ["story", "advertisement"] in descending score order
        p_ad = result["scores"][result["labels"].index(labels[0])]
        p_story = result["scores"][result["labels"].index(labels[1])]

        # add log probabilities
        total_ad_log += math.log(p_ad + 1e-9)
        total_story_log += math.log(p_story + 1e-9)

    # pick which label is best overall
    if total_ad_log > total_story_log:
        return (labels[0], total_ad_log)
    else:
        return (labels[1], total_story_log)

# 4) Compare no-split vs. best single-split
# -- (a) No-split: treat entire chunk as one label
#print(sentences)
a ="""25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club."""

whole_label, whole_score = classify_subchunk(a)

# -- (b) Single-split: try every boundary between sentences
best_split_score = float("-inf")
best_split = None  # store index of the best split
best_left_label = None
best_right_label = None

for w in range(1, len(sentences)):  # possible split points
    left = sentences[:w]
    right = sentences[w:]

    left_label, left_score = classify_subchunk(left,labels)
    right_label, right_score = classify_subchunk(right,labels)

    total_score = left_score + right_score

    if total_score > best_split_score:
        best_split_score = total_score
        best_split = w
        best_left_label = left_label
        best_right_label = right_label

print(f"No-split label: {whole_label}, log-score = {whole_score:.3f}")
if best_split:
    print(f"Best single-split at sentence {best_split}:")
    print(f" Left label: {best_left_label}")
    print(f" Right label: {best_right_label}")
    print(f" Total two-chunk log-score = {best_split_score:.3f}")

# 5) Decide final chunking
if best_split_score > whole_score:
    # We do the split
    final_left = sentences[:best_split]
    final_right = sentences[best_split:]

    print("\nSPLIT CHOSEN:")
    print("LEFT SUB-CHUNK (labeled as", best_left_label, "):")
    for s in final_left:
        print(" -", s)
    print("\nRIGHT SUB-CHUNK (labeled as", best_right_label, "):")
    for s in final_right:
        print(" -", s)
else:
    # Keep chunk as one
    print("\nNO SPLIT CHOSEN. The entire chunk is labeled:", whole_label)
    for s in sentences:
        print(" -", s)

NameError: name 'sentences' is not defined

# The DP approach

In [ ]:
#classifier = pipeline("zero-shot-classification", model="microsoft/deberta-large-mnli")


In [ ]:
segment_text_viterbi(text, labels, classifier)

[('About FLASH SALE!', 'ad'),
 ('25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!', 'ad'),
 ("25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.",
  'ad'),
 ("And that's why we founded The CRNA Club.", 'about us'),
 ('Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.',
  'about us'),
 ('We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.',
  'about us'),
 ('Community focused.', 'about us'),
 ('Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.',
  'about us'),
 ('We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.',
  'about us'),
 ("We don't do normal.", 'about us'),
 ('We overdeliver.', 'about us'),
 ('From the way we teach, to the tools we prov

s

[('spam', 'about us')]
0

Corrected split:
spam: 25% OFF NEW TOOLKITS - USE CODE SAVE25
about us: We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club.
Score: -0.004864884473223025
-----

===== FINAL CORRECTED SEGMENTATION =====
[spam] About FLASH SALE!
[spam] 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
[spam] 25% OFF NEW TOOLKITS - USE CODE SAVE25
[about us] We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club.
[about us] And that's why we founded The CRNA Club.
[about us] Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
[about us] We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
[about us] Community focused.
[about us] Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.
[about us] We love education, resear

In [ ]:
import math
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline

# 1) DOWNLOAD NLTK TOKENIZER
nltk.download("punkt")

# 2) SET UP ZERO-SHOT CLASSIFIER
#classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 3) YOUR TEXT
text = """About FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club. Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner. We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession. Community focused. Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia. We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams. We don't do normal. We overdeliver. From the way we teach, to the tools we provide, we believe in standing out and doing things differently. We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods. We love good humans. We believe in treating people with respect and kindness. From strangers on the Internet, to our SRNAs and fellow colleagues, we practice the Golden Rule. We value transparency and doing the right thing, even when nobody is looking. We promise to bring inclusivity, openness and acceptance to every interaction. For the CRNA school applicant, helpful articles and resources. Check out our most popular applicant resource, The CRNA School Database. Applying to CRNA school? Our digital courses will take you further, faster. And that's why we founded The CRNA Club.    """


# CHOOSE YOUR LABELS
labels = ["spam", "about us"]
# (Feel free to try synonyms like "promotional" vs. "educational")
def segment_text_viterbi(text, labels, classifier, smoothness=0.5, spam_th=0.5,batch_size=32):
    """
    Uses a Viterbi-like dynamic programming approach to label segments of text.
    Low spam_th => more aggressively labeling segments as labels[1] if p(labels[0]) < spam_th.

    Parameters
    ----------
    text : str
        The input text to segment and classify.
    labels : list
        A two-element list of label names, e.g. ["AD", "ST"].
    classifier : function
        A model or pipeline that returns classification results with .scores and .labels fields.
    smoothness : float
        A penalty for switching labels from segment to segment in the Viterbi transitions.
    spam_th : float
        A threshold below which we “push” the probability away from labels[0].
        For instance, if p(labels[0]) < spam_th, we favor labels[1].

    Returns
    -------
    labeled_segments : list of tuples
        Each tuple is (segment_text, assigned_label).
    """


    nltk.download("punkt", quiet=True)

    # 1) Tokenize into segments
    segments = sent_tokenize(text)

    # 2) Compute emission probabilities
    #    We'll store probabilities as list of (p_label0, p_label1) for each segment
    probs = [(0.0, 0.0)] * len(segments)  # pre-allocate

    # Get batch results for all segments
    batch_results = classifier(segments, labels, batch_size=batch_size)

    for i, (seg, result) in enumerate(zip(segments, batch_results)):
        seg = seg.strip()
        if not seg:
            # If the segment is empty, keep default (0.0, 0.0) or handle differently if you prefer
            continue

        # Extract the probability assigned to each label
        p_ad = result["scores"][result["labels"].index(labels[0])]  # e.g., label0
        p_st = result["scores"][result["labels"].index(labels[1])]  # e.g., label1

        # --- THRESHOLD ADJUSTMENT ---
        #
        # If p_ad < spam_th, push the distribution toward label1.
        # One simple approach is to force p_ad to be extremely small
        # so that label1 is heavily favored in the final path.
        # (You can make this more subtle if you prefer.)
        #
        if p_ad < spam_th:
            p_ad = 1e-9
            p_st = 1.0 - p_ad

        # Store
        probs[i] = (p_ad, p_st)

    # 3) Define transition cost function
    #    Returns 0.0 if same label, else -smoothness
    def transition_cost(prev_label, curr_label):
        return 0.0 if prev_label == curr_label else -smoothness

    # 4) Convert to log probabilities
    #    (avoid log(0) by adding a small epsilon)
    eps = 1e-9
    log_probs = [(math.log(ad + eps), math.log(st + eps)) for (ad, st) in probs]

    n = len(segments)

    # 5) Initialize Viterbi DP arrays
    # dp[i][0] = best log-prob. ending at segment i with label0
    # dp[i][1] = best log-prob. ending at segment i with label1
    dp = [[float('-inf')] * 2 for _ in range(n)]
    bp = [[None] * 2 for _ in range(n)]  # back-pointer

    # Base case
    dp[0][0] = log_probs[0][0]
    dp[0][1] = log_probs[0][1]

    # 6) Fill DP
    for i in range(1, n):
        for curr_label in (0, 1):
            for prev_label in (0, 1):
                score = (
                    dp[i - 1][prev_label]
                    + transition_cost(prev_label, curr_label)
                    + log_probs[i][curr_label]
                )
                if score > dp[i][curr_label]:
                    dp[i][curr_label] = score
                    bp[i][curr_label] = prev_label

    # 7) Backtrack to find best path
    final_label = 0 if dp[n - 1][0] > dp[n - 1][1] else 1
    best_path = [final_label]

    for i in range(n - 1, 0, -1):
        best_path.append(bp[i][best_path[-1]])
    best_path.reverse()

    # 8) Pair segments with assigned labels
    label_map = {0: labels[0], 1: labels[1]}
    labeled_segments = [(segments[i].strip(), label_map[best_path[i]]) for i in range(n)]

    return labeled_segments




def print_segmented_text(labeled_segments):

    ads = [seg for seg, lbl in labeled_segments if lbl == labels[0]]
    stories = [seg for seg, lbl in labeled_segments if lbl == labels[1] ]

    print("===== ADVERTISEMENT SEGMENTS =====")
    for seg in ads:
        print("-", seg)

    print("\n===== STORY SEGMENTS =====")
    for seg in stories:
        print("-", seg)

def get_switching_strings(labeled_segments):

    best_path = [seg[1] for seg in labeled_segments]
    segments = [seg[0] for seg in labeled_segments]
    """
    When best_path[i] != best_path[i+1], combine segments[i] + " " + segments[i+1].
    Returns a list of these combined strings.
    """
    switching_strings = []
    switching_labels = []
    switching_indices = []
    n = len(segments)
    if n <= 1:
        return switching_strings,switching_labels,switching_indices

    for i in range(n - 1):
        if best_path[i] != best_path[i+1]:
            combined = segments[i] + " " + segments[i+1]
            #labels
            left_label = best_path[i]
            right_label = best_path[i+1]

            switching_strings.append(combined)
            switching_labels.append((left_label, right_label))
            switching_indices.append(i)

    return switching_strings, switching_labels,switching_indices

import math

def best_split_batch(chunk, left_label, right_label, classifier, all_labels):
    """
    Find the best split for 'chunk' at each possible word boundary [1..N-1],
    forcing the left sub-chunk to 'left_label' and the right sub-chunk to 'right_label',
    but we do it in a single (or batched) classifier call to leverage GPU.

    Returns
    -------
    (best_split_idx, best_score, best_left_str, best_right_str)
      best_split_idx : int or None
      best_score : float (the maximum total_log among all splits)
      best_left_str, best_right_str : str
    """
    words = chunk.split()
    n = len(words)

    # If there's no meaningful way to split (e.g. n < 2),
    # fallback to "all chunk" on left_label. You can pick a default or re-check probabilities.
    if n < 2:
        # single call or skip. For demonstration:
        result = classifier(chunk, all_labels)
        prob = result["scores"][result["labels"].index(left_label)]
        logp = math.log(prob + 1e-9)
        return (None, logp, chunk, "")

    # ----------------------------------------------------------------
    # 1) Generate all sub-chunks:
    #    For i in [1..n-1], left_str = words[:i], right_str = words[i:].
    #    We'll gather them in a single list so we can do a single classifier() call in one or more big batches.
    # ----------------------------------------------------------------
    left_subchunks = []
    right_subchunks = []
    for i in range(1, n):
        left_subchunks.append(" ".join(words[:i]))
        right_subchunks.append(" ".join(words[i:]))

    # We'll have len(words) - 1 left_subchunks, and the same number of right_subchunks.
    # We'll combine them into a single list, so we can classify them all in one go:
    combined_subchunks = left_subchunks + right_subchunks
    # Example: we have 2*(n-1) total sub-chunks.

    # ----------------------------------------------------------------
    # 2) Classify them in one (or few) big batches on the GPU.
    #    The classifier presumably returns a list of dicts with "scores" and "labels".
    # ----------------------------------------------------------------
    results = classifier(combined_subchunks, all_labels)
    # Make sure your classifier can handle large batches or break it up if needed.

    # Now, the first (n-1) results correspond to left_subchunks,
    # the next (n-1) correspond to right_subchunks.
    left_results = results[: (n - 1)]
    right_results = results[(n - 1) :]

    # ----------------------------------------------------------------
    # 3) Compute log(prob) for the assigned label for each sub-chunk.
    #    - left_subchunks must be forced to left_label
    #    - right_subchunks must be forced to right_label
    # ----------------------------------------------------------------
    left_logs = []
    for r in left_results:
        p_left = r["scores"][r["labels"].index(left_label)]
        left_logs.append(math.log(p_left + 1e-9))

    right_logs = []
    for r in right_results:
        p_right = r["scores"][r["labels"].index(right_label)]
        right_logs.append(math.log(p_right + 1e-9))

    # ----------------------------------------------------------------
    # 4) Combine left_logs[i] + right_logs[i] to find the best split
    # ----------------------------------------------------------------
    best_score = float("-inf")
    best_split_idx = None
    best_left_str = None
    best_right_str = None

    for i in range(n - 1):
        total_log = left_logs[i] + right_logs[i]
        if total_log > best_score:
            best_score = total_log
            best_split_idx = i + 1  # i+1 words go to the left
            best_left_str = left_subchunks[i]
            best_right_str = right_subchunks[i]

    return best_split_idx, best_score, best_left_str, best_right_str


def fix_splits(
    labeled_segments,
    switching_labels,
    switching_strings,
    switching_indices,
    classifier,
    labels
):
    """
    Attempt to fix segments where a 'switch' of labels is detected mid-segment.
    This version ensures we skip trivial edge splits. We do NOT merge adjacent
    segments if they share the same label.

    Parameters
    ----------
    labeled_segments : list of tuples (segment_text, label)
    switching_labels : list of tuples (left_label, right_label)
    switching_strings : list of strings (the text chunk for each switch)
    switching_indices : list of segment indices to fix
    classifier : function to evaluate splits
    labels : [label0, label1]
    """

    # 1) Create a dictionary to store the "split replacements"
    #    for segments that need to be fixed.
    split_replacements = {}

    # 2) Fix each "switching" chunk by trying best_split()
    for i, s in enumerate(switching_strings):
        left_label, right_label = switching_labels[i]
        seg_index = switching_indices[i]  # which segment to replace

        split_idx, score = best_split(s, left_label, right_label, classifier, labels)
        words = s.split()

        # If the best split is at 0 or end, skip splitting and assign full chunk to one label.
        if split_idx <= 0 or split_idx >= len(words):
            # You can pick whichever label you prefer; here's an example using left_label:
            entire_label = left_label
            split_replacements[seg_index] = [(s, entire_label)]
        else:
            # Valid split: slice the text into left & right sub-chunks
            left_str = " ".join(words[:split_idx])
            right_str = " ".join(words[split_idx:])

            split_replacements[seg_index] = [
                (left_str, left_label),
                (right_str, right_label)
            ]

    # 3) Rebuild the final segmentation.
    corrected_segments = []
    for i, (seg_text, seg_label) in enumerate(labeled_segments):
        if i in split_replacements:
            # Replace with the two (or one) corrected sub-chunks
            corrected_segments.extend(split_replacements[i])
        else:
            # Keep the original segment if no replacement
            corrected_segments.append((seg_text, seg_label))

    return corrected_segments





### all together


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

labeled_segments = segment_text_viterbi(text, labels, classifier,smoothness=0.25)
print_segmented_text(labeled_segments)
switching_strings,switching_labels, switching_indices = get_switching_strings(labeled_segments)
corrected_segments = fix_splits(labeled_segments,switching_labels,switching_strings,switching_indices, classifier,labels)

print("\n===== FINAL CORRECTED SEGMENTATION =====")
for text_seg, label_seg in corrected_segments:
    print(f"[{label_seg}] {text_seg}")

===== ADVERTISEMENT SEGMENTS =====
- About FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
- Check out our most popular applicant resource, The CRNA School Database.

===== STORY SEGMENTS =====
- And that's why we founded The CRNA Club.
- Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
- We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
- Community focused.
- Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.
- We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
- We don't do normal.
- We overdeliver.
- From the way we teach, to the tools we provide, we believe in standing out

In [ ]:
def segment_text(text, labels, classifier,smoothness=0.5,spam_th=0.3,batch_size=32):
  labeled_segments = segment_text_viterbi(text, labels, classifier,smoothness=smoothness,spam_th=spam_th,batch_size=batch_size)
  #print_segmented_text(labeled_segments)
  switching_strings,switching_labels, switching_indices = get_switching_strings(labeled_segments)

  corrected_segments = fix_splits(labeled_segments,switching_labels,switching_strings,switching_indices, classifier,labels)
  return corrected_segments

print("\n===== FINAL CORRECTED SEGMENTATION =====")
for text_seg, label_seg in corrected_segments:
    print(f"[{label_seg}] {text_seg}")


===== FINAL CORRECTED SEGMENTATION =====
[spam] About FLASH SALE!
[spam] 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
[spam] 25% OFF NEW TOOLKITS - USE CODE SAVE25
[about us] We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club.
[about us] And that's why we founded The CRNA Club.
[about us] Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
[about us] We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
[about us] Community focused.
[about us] Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.
[about us] We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
[about us] We don't do normal.
[about us] We overdeliver.
[about us] From the way we teach,

# Run through the entire dataset

In [ ]:
# loop through df rows
labels = ["ad","our values"]
for i,r in df.iterrows():
  print(i)
  if i < 0:
    pass
  else:
    corrected_segments = segment_text(r["text"], labels, classifier,smoothness=0.3,spam_th=0.3,batch_size=128)
    purged_text= [text_seg for text_seg, label_seg in corrected_segments if label_seg == labels[0]]
    purged_text_str = "\n".join(purged_text)

    clean_text = [text_seg for text_seg, label_seg in corrected_segments if label_seg == labels[1]]
    clean_text_str = "\n".join(clean_text)

    # put it in df["clean_text"]
    df.at[i, 'clean_text'] = clean_text_str
    df.at[i, 'purged_text'] = purged_text_str







0
1
2
3
4
5
6
7
8
9
10
11
12
13


KeyboardInterrupt: 

In [ ]:
print(len(df))
print(len(df[df['clean_text'] != '']))

32486
139


In [ ]:
# display rows where clean_text column isn't empty
display(df[df['clean_text'] != ''])

for i,r in df[df['clean_text'] != ''].iterrows():
  print(f"======================clean=================")
  print(r["clean_text"])
  print(f"======================purged=================")
  print(r["purged_text"])


,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
1,https://active-silver.co.uk/pages/about-active...,Country/regionCountry/region ...,5.0,7,active-silver.co.uk,active-silver.co.uk/pages/about-active-silver,US,en,shopify,Country/regionCountry/region ...,and on animals too. Explore our Customer Testi...
2,https://smartstore24.org/pages/about-us,About Us Summer Sale - 50% OFF + FREE Shipping...,1.0,4,smartstore24.org,smartstore24.org/pages/about-us,CN,en,shopify,taste and occasion. We thoroughly check the qu...,About Us Summer Sale - 50% OFF + FREE Shipping...
3,https://www.kbeautyshop.com.co/quienes-somos,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,1.0,17,kbeautyshop.com.co,kbeautyshop.com.co/quienes-somos,CO,es,tiendanube,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,
4,https://www.infinitymarts.com.br/pages/quem-somos,Quem Somos Frete Grátis para todo o Brasil ...,1.0,13,infinitymarts.com.br,infinitymarts.com.br/pages/quem-somos,BR,pt,shopify,Garantida ou Dinheiro de VoltaPrecisa de ajuda...,Quem Somos Frete Grátis para todo o Brasil Tro...
5,https://httpalirabexpress.com/pages/about-us,"About Us Launched In 2015, Ali Rub Express Is ...",0.0,7,httpalirabexpress.com,httpalirabexpress.com/pages/about-us,US,en,shopify,Consumers.\nSupported By A Wide Range Of Tailo...,"About Us Launched In 2015, Ali Rub Express Is ..."
...,...,...,...,...,...,...,...,...,...,...,...
199,https://shopmixbeautylab.com/pages/about-us,About us FREE STANDARD SHIPPING ON ORDERS $100...,2.0,13,shopmixbeautylab.com,shopmixbeautylab.com/pages/about-us,CA,en,shopify,About us FREE STANDARD SHIPPING ON ORDERS $100...,services makeup and skin related. Please email...
200,https://toonsbridgedairy.com/pages/about-toons...,About Toons Bridge DairyMaking Mediterranean S...,1.0,10,toonsbridgedairy.com,toonsbridgedairy.com/pages/about-toons-bridge-...,US,en,shopify,make cheeses informed by the traditions of the...,About Toons Bridge DairyMaking Mediterranean S...
201,https://bebezarco.com/pages/about,ABOUT Your cart is emptySIGN UP FOR NEWSLETTER...,1.0,9,bebezarco.com,bebezarco.com/pages/about,US,en,shopify,style you won't soon forget. You may have met ...,ABOUT Your cart is emptySIGN UP FOR NEWSLETTER...
202,https://thevettestore.com/pages/about-us,About Us THE VETTE STOREThe Vette Store is ope...,0.0,14,thevettestore.com,thevettestore.com/pages/about-us,US,en,shopify,About Us THE VETTE STOREThe Vette Store is ope...,"of America's favorite sports car, the Chevrole..."


======================clean=================
Country/regionCountry/region                       A powerful natural treatment                     Colloidal Silver is a liquid made of nano sized silver particles, suspended in ultra-pure water.
This natural product can help with a huge variety of health conditions in adults, children and animals.
It’s known to be effective against 650 strains of bacteria, viruses and fungi, and is well-known for its skin healing properties if made correctly.
Colloidal Silver is also the key ingredient in our topical products which means we don’t have to use any nasty artificial preservatives or parabens, so you can rest assured our products are as natural and safe as they can be.
An ancient remedy                     Silver as a natural treatment is nothing new.
As far back as Ancient Greece, they used a silver coin in a barrel of water to stop bacteria growing.
Since then, it has been used throughout the ages to fight infection, in everything from hospit

In [ ]:
# 2) Write desired output to a text file
with open("output.txt", "w", encoding="utf-8") as f:
    for i, r in df[df["clean_text"] != ""].iterrows():
        f.write(f"=======================================\n\t\t\t\t\t\tsample{i}\n=======================================")
        f.write("======================clean=================\n")
        f.write(r["clean_text"] + "\n")  # Already contains \n between segments
        f.write("======================purged=================\n")
        f.write(r["purged_text"] + "\n") # Already contains \n between segments
        f.write(r["text"] + "\n\n")      # Original text

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
import math

nltk.download("punkt", quiet=True)
def safe_sent_tokenize(text):
    """
    Attempt to tokenize text into sentences; if text isn't a valid string,
    catch the TypeError and return an empty list to avoid breaking the code.
    """
    try:
        return sent_tokenize(text)
    except (TypeError, AttributeError):
        # TypeError -> text is float or None
        # AttributeError -> text might be None or weird object
        return []

def batch_classify_segments(df, text_col, labels, classifier, batch_size=32):
    """
    1) Split each row's text into segments (via sent_tokenize).
    2) Classify all segments in large batches, storing the label for each segment.
    3) Rebuild 'clean_text' (label=1) and 'purged_text' (label=0) columns in df.

    Parameters
    ----------
    df : pandas.DataFrame
        The input DataFrame with at least one column containing text.
    text_col : str
        The name of the column in df that has the text to be segmented & classified.
    labels : list of str
        e.g. [label0, label1]. We'll assume label0 => "purged", label1 => "clean".
    classifier : callable
        A function/pipeline that can do batch classification:
          classifier(list_of_texts, list_of_labels, batch_size=...)
        returning a list of dicts with "labels" and "scores".
    batch_size : int
        How many segments to process at once on the GPU.

    Returns
    -------
    df : pandas.DataFrame (modified in place)
        With new columns 'clean_text' and 'purged_text'.
    """

import math
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt", quiet=True)

def safe_sent_tokenize(text):
    """
    Tokenize text into sentences; if text isn't a valid string (float/NaN/etc.),
    catch the TypeError and return an empty list.
    """
    try:
        return sent_tokenize(text)
    except (TypeError, AttributeError):
        return []

def classify_batch_with_threshold(segment_list, labels, classifier, spam_th=0.5, batch_size=32):
    """
    1) Classify all segments in one or more large batches.
    2) Apply threshold logic: if p_label0 < spam_th => forcibly push it to label1.
    3) Return assigned labels (0 or 1).

    segment_list : list of str
    labels       : [label0, label1]
    classifier   : function that takes (list_of_strings, label_list, batch_size=...)
                   and returns list of dicts with "scores", "labels"
    spam_th      : threshold; if p_label0 < spam_th => p_label0=1e-9 => label1 is forced
    """
    results = []
    # We'll store the final assigned labels as integers (0 or 1) in the same order as segment_list.
    assigned_labels = []

    # Step A: batch inference
    for start in range(0, len(segment_list), batch_size):
        batch = segment_list[start : start + batch_size]
        batch_outputs = classifier(batch, labels, batch_size=batch_size)

        # Step B: threshold logic, pick label
        for res in batch_outputs:
            # Probability of label0, label1
            idx0 = res["labels"].index(labels[0])
            idx1 = res["labels"].index(labels[1])
            p0 = res["scores"][idx0]
            p1 = res["scores"][idx1]

            # If p0 < spam_th, force p0 near zero => label1 is chosen
            if p0 < spam_th:
                p0 = 1e-9
                p1 = 1.0 - p0

            # Decide label by max probability
            if p0 > p1:
                assigned_labels.append(0)
            else:
                assigned_labels.append(1)

    return assigned_labels

def batch_classify_segments(df, text_col, labels, classifier, spam_th=0.5, batch_size=32):
    """
    1) Safely tokenize each row's text (catching floats/NaNs).
    2) Classify all segments in big batches with threshold logic.
    3) Rebuild 'clean_text' (label=1) and 'purged_text' (label=0).
    """
    # (row_idx, segment_text)
    segments = []
    # We'll store how many segments each row has, to regroup them later
    segment_counts = []

    # A) Safe tokenization
    for row_i, row in df.iterrows():
        text_val = row[text_col]
        sents = safe_sent_tokenize(text_val)  # returns a list of sentence strings
        segment_counts.append(len(sents))
        for s in sents:
            segments.append((row_i, s))

    if not segments:
        # No valid segments at all
        df["clean_text"] = ""
        df["purged_text"] = ""
        return df

    # B) Classify all segments with threshold logic
    #    We'll do it in bigger batches on the GPU
    all_segment_texts = [seg_text for (_row_i, seg_text) in segments]
    assigned_labels = classify_batch_with_threshold(
        segment_list = all_segment_texts,
        labels       = labels,
        classifier   = classifier,
        spam_th      = spam_th,
        batch_size   = batch_size
    )

    # C) Rebuild clean_text / purged_text
    df["clean_text"] = ""
    df["purged_text"] = ""

    position = 0
    for row_i, row in df.iterrows():
        seg_count = segment_counts[row_i]
        row_segments = segments[position : position + seg_count]
        row_assigned = assigned_labels[position : position + seg_count]
        position += seg_count

        # Separate them by label
        purged_parts = []
        clean_parts  = []
        for (seg_row_i, seg_text), lbl in zip(row_segments, row_assigned):
            if lbl == 0:
                purged_parts.append(seg_text)
            else:
                clean_parts.append(seg_text)

        # Join them back
        df.at[row_i, "purged_text"] = " \n".join(purged_parts)
        df.at[row_i, "clean_text"]  = " \n".join(clean_parts)

    return df



df2 = df.copy().head(10)
df2 = batch_classify_segments(df2, text_col="text", labels=labels, classifier=classifier, batch_size=1024)




In [ ]:
# display rows where clean_text column isn't empty
display(df2[df2['clean_text'] != ''].head(10))

for i,r in df2[df2['clean_text'] != ''].head(10).iterrows():
  print(f"======================clean=================")
  print(r["clean_text"])
  print(f"======================purged=================")
  print(r["purged_text"])


,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
0,https://crafthq.co.uk/pages/about-us,Free delivery on orders over £40Free delivery ...,0.0,7,crafthq.co.uk,crafthq.co.uk/pages/about-us,US,en,shopify,As owners of a successful UK based home fragra...,Free delivery on orders over £40Free delivery ...
1,https://active-silver.co.uk/pages/about-active...,Country/regionCountry/region ...,5.0,7,active-silver.co.uk,active-silver.co.uk/pages/about-active-silver,US,en,shopify,Country/regionCountry/region ...,Explore our Customer Testimonials page for mor...
2,https://smartstore24.org/pages/about-us,About Us Summer Sale - 50% OFF + FREE Shipping...,1.0,4,smartstore24.org,smartstore24.org/pages/about-us,CN,en,shopify,"We thoroughly check the quality of our goods, ...",About Us Summer Sale - 50% OFF + FREE Shipping...
3,https://www.kbeautyshop.com.co/quienes-somos,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,1.0,17,kbeautyshop.com.co,kbeautyshop.com.co/quienes-somos,CO,es,tiendanube,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,
4,https://www.infinitymarts.com.br/pages/quem-somos,Quem Somos Frete Grátis para todo o Brasil ...,1.0,13,infinitymarts.com.br,infinitymarts.com.br/pages/quem-somos,BR,pt,shopify,Nós trabalhamos com o intuito de disponibiliza...,Quem Somos Frete Grátis para todo o Brasil ...
5,https://httpalirabexpress.com/pages/about-us,"About Us Launched In 2015, Ali Rub Express Is ...",0.0,7,httpalirabexpress.com,httpalirabexpress.com/pages/about-us,US,en,shopify,Supported By A Wide Range Of Tailored Marketin...,"About Us Launched In 2015, Ali Rub Express Is ..."
6,https://gigiszone.com/pages/about-us,About us Awkwardness gives me great comfort. I...,2.0,81,gigiszone.com,gigiszone.com/pages/about-us,US,en,shopify,About us Awkwardness gives me great comfort. \...,I don’t really know how to do casual clothes.G...
7,https://minisan.com/pages/about,"SUMMER SALES -30% | FREE SHIPPING OVER €29,90 ...",5.0,14,minisan.com,minisan.com/pages/about,US,en,shopify,"SUMMER SALES -30% | FREE SHIPPING OVER €29,90 ...","Japanese Konbini stores ""コンビニ"" are a great exa..."
8,https://chilithom.com/pages/about-chili-thom,CartYour cart is empty. Continue browsing ...,4.0,10,chilithom.com,chilithom.com/pages/about-chili-thom,SG,en,shopify,CartYour cart is empty. \nUntil his untimely d...,Continue browsing Shipping & taxes calculate...
9,https://lushvirginhair.com/pages/about-us,About Us Your cart is currently empty!Continue...,0.0,13,lushvirginhair.com,lushvirginhair.com/pages/about-us,US,en,shopify,About Us Your cart is currently empty!Continue...,HAIR LOSS SOLUTIONS We offer a bespoke custom...


======================clean=================
As owners of a successful UK based home fragrance brand, we know what it takes.
======================purged=================
Free delivery on orders over £40Free delivery on orders over £40For Candle Makers by Candle MakersHere at Craft HQ we thoroughly test all fragrance oils to ensure they perform as well as can be in candles, wax melts and lots of other fragranced products.Starting your own candle and wax melt business is hard work. 
Not only have you got to perfect your craft and create the best candles and wax melts possible, but you also need to market yourself and get customers! 
We have created Craft HQ to help other candle and wax melt businesses start, grow and thrive through the use of our unique fragrance oils and marketing advice.This site is protected by reCAPTCHA and the Google Privacy Policy and Terms of Service apply.Unit 7 Boss Hall RoadIpswich, IP1 5BNMon - Fri, 8:00am - 4:00pm
======================clean=================

In [ ]:
display(df.head(10))

,about_url,text,images_count,archivo,url,url_about,country_code,language,server,clean_text,purged_text
0,https://crafthq.co.uk/pages/about-us,Free delivery on orders over £40Free delivery ...,0.0,7,crafthq.co.uk,crafthq.co.uk/pages/about-us,US,en,shopify,,Free delivery on orders over £40Free delivery ...
1,https://active-silver.co.uk/pages/about-active...,Country/regionCountry/region ...,5.0,7,active-silver.co.uk,active-silver.co.uk/pages/about-active-silver,US,en,shopify,Country/regionCountry/region ...,and on animals too. Explore our Customer Testi...
2,https://smartstore24.org/pages/about-us,About Us Summer Sale - 50% OFF + FREE Shipping...,1.0,4,smartstore24.org,smartstore24.org/pages/about-us,CN,en,shopify,taste and occasion. We thoroughly check the qu...,About Us Summer Sale - 50% OFF + FREE Shipping...
3,https://www.kbeautyshop.com.co/quienes-somos,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,1.0,17,kbeautyshop.com.co,kbeautyshop.com.co/quienes-somos,CO,es,tiendanube,Quiénes Somos ¿Quiénes Somos?Bienvenidos a K-B...,
4,https://www.infinitymarts.com.br/pages/quem-somos,Quem Somos Frete Grátis para todo o Brasil ...,1.0,13,infinitymarts.com.br,infinitymarts.com.br/pages/quem-somos,BR,pt,shopify,Garantida ou Dinheiro de VoltaPrecisa de ajuda...,Quem Somos Frete Grátis para todo o Brasil Tro...
5,https://httpalirabexpress.com/pages/about-us,"About Us Launched In 2015, Ali Rub Express Is ...",0.0,7,httpalirabexpress.com,httpalirabexpress.com/pages/about-us,US,en,shopify,Consumers.\nSupported By A Wide Range Of Tailo...,"About Us Launched In 2015, Ali Rub Express Is ..."
6,https://gigiszone.com/pages/about-us,About us Awkwardness gives me great comfort. I...,2.0,81,gigiszone.com,gigiszone.com/pages/about-us,US,en,shopify,About us Awkwardness gives me great comfort.\n...,know how to do casual clothes.Get complimentar...
7,https://minisan.com/pages/about,"SUMMER SALES -30% | FREE SHIPPING OVER €29,90 ...",5.0,14,minisan.com,minisan.com/pages/about,US,en,shopify,"SUMMER SALES -30% | FREE SHIPPING OVER €29,90 ...",Asian countries!INFOCOMPANYBIONIV GmbHVAT/MwSt...
8,https://chilithom.com/pages/about-chili-thom,CartYour cart is empty. Continue browsing ...,4.0,10,chilithom.com,chilithom.com/pages/about-chili-thom,SG,en,shopify,CartYour cart is empty. Continue browsing Ship...,at checkoutAboutThe ArtistChili Thom was one o...
9,https://lushvirginhair.com/pages/about-us,About Us Your cart is currently empty!Continue...,0.0,13,lushvirginhair.com,lushvirginhair.com/pages/about-us,US,en,shopify,About Us Your cart is currently empty!Continue...,"from our beautiful hair pieces, Wigs to hair i..."
